<a href="https://colab.research.google.com/github/DagmarMpheio/fruit_ripeness_yolo8/blob/main/Fruit_Ripeness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8

# **DETENÇÃO DO ESTADO DE MATURAÇÃO DAS FRUTAS**

# Instalar Utralytics

In [50]:
!nvidia-smi

Sun Dec  1 20:27:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [51]:
import os
import threading
HOME = os.getcwd()
print(HOME)

/content


In [52]:
!pip install ultralytics

from IPython import display
display.clear_output()
!yolo checks

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.0/112.6 GB disk)

OS                  Linux-6.1.85+-x86_64-with-glibc2.35
Environment         Colab
Python              3.10.12
Install             pip
RAM                 12.67 GB
Disk                33.0/112.6 GB
CPU                 Intel Xeon 2.00GHz
CPU count           2
GPU                 Tesla T4, 15102MiB
GPU count           1
CUDA                12.1

numpy               ✅ 1.26.4>=1.23.0
numpy               ✅ 1.26.4<2.0.0; sys_platform == "darwin"
matplotlib          ✅ 3.8.0>=3.3.0
opencv-python       ✅ 4.10.0.84>=4.6.0
pillow              ✅ 11.0.0>=7.1.2
pyyaml              ✅ 6.0.2>=5.3.1
requests            ✅ 2.32.3>=2.23.0
scipy               ✅ 1.13.1>=1.4.1
torch               ✅ 2.5.1+cu121>=1.8.0
torch               ✅ 2.5.1+cu121!=2.4.0,>=1.8.0; sys_platform == "win32"
torchvision         ✅ 0.20.1+cu121>=0.9.0
tqdm                ✅ 4.67.1>=4.64.0
psu

# Criar a divisão de dados para treino e validação

In [53]:
# importar as bibliotecas necessarios
import os
import shutil
import threading
import time
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm # Para mostrar barra de progresso

In [54]:
# conectando ao google drive

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#pip install -r requirements.txt

In [55]:
train_path_img="./yolov8_detecao_maturacao_frutas/images/train"
train_path_label="./yolov8_detecao_maturacao_frutas/labels/train"
val_path_img="./yolov8_detecao_maturacao_frutas/images/val"
val_path_label="./yolov8_detecao_maturacao_frutas/labels/val"
test_path="./yolov8_detecao_maturacao_frutas/test"

In [56]:
'''
Dividir o dataset em treino e teste e cria o train.txt e test.txt com
o respectivo caminho das imagens em cada pasta
'''

def train_test_split(path, neg_path=None, split=0.2):
    print("------ PROCESSO INICIADO ------")


    # Criar diretórios
    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    # Extensões permitidas
    allowed_extensions = {'.jpg', '.jpeg', '.png'}

    # Listar imagens com as extensões permitidas
    files = list(set(
        [os.path.splitext(name)[0] for name in os.listdir(path) if os.path.splitext(name)[1].lower() in allowed_extensions]
    ))
    print(f"--- Esta pasta tem no total {len(files)} imagens ---")

    # Shuffle
    random.seed(42)
    random.shuffle(files)

    # Dividir treino e teste
    test_size = int(len(files) * split)
    train_files = files[:-test_size]
    test_files = files[-test_size:]

    # Criar arquivos train.txt e test.txt
    with open("train.txt", "w") as train_file, open("test.txt", "w") as test_file:
        # Mover arquivos de treino
        for filex in tqdm(train_files, desc="Treino"):
            for ext in allowed_extensions:
                img_path = os.path.join(path, filex + ext)
                if os.path.exists(img_path):
                    shutil.copy2(img_path, os.path.join(train_path_img, filex + ext))
                    train_file.write(os.path.join(train_path_img, filex + ext) + '\n')
                    break
            label_path = os.path.join(path, filex + '.txt')
            if os.path.exists(label_path):
                shutil.copy2(label_path, os.path.join(train_path_label, filex + '.txt'))

        # Mover arquivos de teste
        for filex in tqdm(test_files, desc="Teste"):
            for ext in allowed_extensions:
                img_path = os.path.join(path, filex + ext)
                if os.path.exists(img_path):
                    shutil.copy2(img_path, os.path.join(val_path_img, filex + ext))
                    test_file.write(os.path.join(val_path_img, filex + ext) + '\n')
                    break
            label_path = os.path.join(path, filex + '.txt')
            if os.path.exists(label_path):
                shutil.copy2(label_path, os.path.join(val_path_label, filex + '.txt'))

    print(f"------ Treino: {len(train_files)} imagens, Teste: {len(test_files)} imagens ------")

    # Adicionar imagens negativas, se especificado
    if neg_path:
        neg_images = list(set(
            [os.path.splitext(name)[0] for name in os.listdir(neg_path) if os.path.splitext(name)[1].lower() in allowed_extensions]
        ))
        for filex in tqdm(neg_images, desc="Negativas"):
            for ext in allowed_extensions:
                img_path = os.path.join(neg_path, filex + ext)
                if os.path.exists(img_path):
                    shutil.copy2(img_path, os.path.join(train_path_img, filex + ext))
                    break
        print(f"------ Total de {len(neg_images)} imagens negativas adicionadas ao treino ------")

    print("------ TAREFA CONCLUIDA -------")

# Exemplo de uso
train_test_split('/content/drive/MyDrive/dataset-frutas/')



### for label_tag
train_test_split('/content/drive/MyDrive/dataset-frutas/') ### sem imagens negativas

------ PROCESSO INICIADO ------
--- Esta pasta tem no total 76 imagens ---


Treino:   0%|          | 0/61 [00:00<?, ?it/s]

Teste:   0%|          | 0/15 [00:00<?, ?it/s]

------ Treino: 61 imagens, Teste: 15 imagens ------
------ TAREFA CONCLUIDA -------
------ PROCESSO INICIADO ------
--- Esta pasta tem no total 76 imagens ---


Treino:   0%|          | 0/61 [00:00<?, ?it/s]

Teste:   0%|          | 0/15 [00:00<?, ?it/s]

------ Treino: 61 imagens, Teste: 15 imagens ------
------ TAREFA CONCLUIDA -------


In [57]:
import ultralytics

ultralytics.checks()

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.0/112.6 GB disk)


# Treinar o modelo

In [58]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/drive/MyDrive/yolov8/dataset-frutas.yaml epochs=100 patience=20 batch=16 imgsz=640 project=/content/drive/MyDrive/yolov8/training_results name='fruits'

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/yolov8/dataset-frutas.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=fruits3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

### Copiar algumas images para teste

In [59]:
!cp -av '/content/yolov8_detecao_maturacao_frutas/images/val/' '/content/drive/MyDrive/yolov8/test_images_fruits'

'/content/yolov8_detecao_maturacao_frutas/images/val/' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val'
'/content/yolov8_detecao_maturacao_frutas/images/val/img_0_516.jpeg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/img_0_516.jpeg'
'/content/yolov8_detecao_maturacao_frutas/images/val/maçã verde_1.jpeg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/maçã verde_1.jpeg'
'/content/yolov8_detecao_maturacao_frutas/images/val/img_0_232.jpeg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/img_0_232.jpeg'
'/content/yolov8_detecao_maturacao_frutas/images/val/img_0_662.jpeg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/img_0_662.jpeg'
'/content/yolov8_detecao_maturacao_frutas/images/val/img_0_51.jpeg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/img_0_51.jpeg'
'/content/yolov8_detecao_maturacao_frutas/images/val/img_0_109.jpeg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/img_0_109.jpeg'
'/content/yolov8_detecao_ma

# Outra avaliação do modelo

In [60]:
!yolo task=detect mode=val model='/content/drive/MyDrive/yolov8/training_results/fruits3/weights/best.pt' data=/content/drive/MyDrive/yolov8/dataset-frutas.yaml

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,849,024 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/yolov8_detecao_maturacao_frutas/labels/val.cache... 20 images, 0 backgrounds, 0 corrupt: 100% 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:01<00:00,  1.20it/s]
                   all         20         26      0.702      0.779      0.894       0.81
          Banana Verde          2          2      0.859          1      0.995      0.995
    Banana Semi Madura          1          1      0.797          1      0.995      0.995
         Banana Madura          2          2      0.884          1      0.995      0.995
          Banana Podre          2          2      0.337          1      0.995       0.92
         Morango Podre          1          1       0.89          1      0.995      0.895
        Morango Maduro          1    

In [61]:
#copiar diretorios
!cp -r /content/runs/detect/val /content/drive/MyDrive/yolov8/val-100-epocas-batch-16-patience-20-modelo-m

# Inferência

In [62]:
!yolo task=detect mode=predict model='/content/drive/MyDrive/yolov8/training_results/fruits3/weights/best.pt' conf=0.55 save=True source=/content/drive/MyDrive/yolov8/test_images_fruits/val

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,849,024 parameters, 0 gradients, 78.7 GFLOPs

image 1/20 /content/drive/MyDrive/yolov8/test_images_fruits/val/7b3ee0c2e3e0f2a142e49a06f51f5581.jpg: 256x640 1 Banana Verde, 1 Banana Semi Madura, 1 Banana Madura, 102.8ms
image 2/20 /content/drive/MyDrive/yolov8/test_images_fruits/val/banana madura_12.jpeg: 480x640 1 Banana Madura, 69.5ms
image 3/20 /content/drive/MyDrive/yolov8/test_images_fruits/val/banana podre ou estragada_56.jpeg: 576x640 1 Banana Podre, 70.0ms
image 4/20 /content/drive/MyDrive/yolov8/test_images_fruits/val/banana podre ou estragada_6.jpg: 608x640 1 Banana Podre, 68.2ms
image 5/20 /content/drive/MyDrive/yolov8/test_images_fruits/val/images2.jpeg: 576x640 1 Maçã Podre, 36.6ms
image 6/20 /content/drive/MyDrive/yolov8/test_images_fruits/val/img_0_109.jpeg: 416x640 1 Banana Verde, 106.1ms
image 7/20 /content/drive/MyDrive/yolov8/test_images_fruits/val/i

In [63]:
#copiar diretorios
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output-50-epocas-batch-16-patience-50-modelo-m

# Inferência em videos

In [ ]:
!yolo task=detect mode=predict model='/content/drive/MyDrive/yolov8/training_results/fruits2/weights/best.pt' conf=0.55 save=True source=/content/drive/MyDrive/yolov8/test_videos_frutas

Streaming output truncated to the last 5000 lines.
video 3/4 (5153/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.2ms
video 3/4 (5154/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.3ms
video 3/4 (5155/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.5ms
video 3/4 (5156/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.5ms
video 3/4 (5157/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.7ms
video 3/4 (5158/7704) /content/dri

In [ ]:
#copiar diretorios com resultados de inferencia em videos
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output-3-epocas-batch-16-patience-10-modelo-m

In [ ]:
#exluir diretorios nao vazios
!rm -r /content/runs

In [ ]:
!python --version